In [ ]:
# Verificar e instalar dependencias necesarias
import subprocess
import sys

def verificar_e_instalar_dependencias():
    """Verificar e instalar dependencias necesarias para el notebook."""
    dependencias = [
        'opencv-python',
        'numpy', 
        'pandas',
        'nibabel',
        'pydicom',
        'requests'
    ]
    
    dependencias_instaladas = []
    dependencias_faltantes = []
    
    print("Verificando dependencias...")
    
    for dep in dependencias:
        try:
            __import__(dep.replace('-', '_') if '-' in dep else dep)
            dependencias_instaladas.append(dep)
            print(f"+ {dep}: Disponible")
        except ImportError:
            dependencias_faltantes.append(dep)
            print(f"- {dep}: Faltante")
    
    if dependencias_faltantes:
        print(f"\nInstalando dependencias faltantes: {', '.join(dependencias_faltantes)}")
        for dep in dependencias_faltantes:
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", dep])
                print(f"+ {dep}: Instalado exitosamente")
            except subprocess.CalledProcessError as e:
                print(f"- Error instalando {dep}: {e}")
    
    print(f"\nResumen:")
    print(f"- Dependencias verificadas: {len(dependencias)}")
    print(f"- Ya disponibles: {len(dependencias_instaladas)}")
    print(f"- Instaladas ahora: {len(dependencias_faltantes)}")
    
    if dependencias_faltantes:
        print("\nIMPORTANTE: Puede ser necesario reiniciar el kernel para que los cambios tengan efecto.")
        print("   Ve a 'Kernel' > 'Restart' en el menú de Jupyter después de ejecutar esta celda.")

# Ejecutar verificación automáticamente
verificar_e_instalar_dependencias()

In [ ]:
# Verificar importación de OpenCV específicamente
try:
    import cv2
    print(f"+ OpenCV importado exitosamente")
    print(f"  Versión: {cv2.__version__}")
    
    # Probar funcionalidad básica
    import numpy as np
    test_image = np.zeros((100, 100, 3), dtype=np.uint8)
    print(f"  Prueba básica de creación de imagen: OK")
    
    print(f"\nOpenCV está listo para usar!")
    
except ImportError as e:
    print(f"- Error al importar OpenCV: {e}")
    print(f"\nSolucion sugerida:")
    print(f"   1. Ejecuta la celda anterior para instalar dependencias")
    print(f"   2. Ve a 'Kernel' > 'Restart' para reiniciar el kernel")
    print(f"   3. Vuelve a ejecutar todas las celdas desde el inicio")
    
except Exception as e:
    print(f"- Error inesperado con OpenCV: {e}")
    print(f"   OpenCV está instalado pero puede tener problemas")

# Sistema de Gestión de Imágenes Médicas - Trabajo Final Grupo 4

Este notebook proporciona una interfaz interactiva para el Sistema de Gestión de Imágenes Médicas.

## Opciones del Sistema

1. Registrar Nueva Imagen
2. Ver Imágenes Registradas  
3. Modificar Metadatos de Imagen
4. Eliminar Imagen
5. Buscar Imágenes (extra)
6. Cargar Dataset de Zenodo (extra)
7. Salir

Ejecute las celdas en orden para utilizar el sistema de manera interactiva.

## 1. Configuración Inicial

Importar librerías y configurar el sistema

In [1]:
# Importar librerías necesarias
import sys
import os
import numpy as np
from datetime import datetime
from typing import Dict, List, Optional, Any

# Configurar path del proyecto
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Importar módulos del sistema
from medical_image_manager import ImageManager, DataLoader, MedicalImage, ImageMetadata
from medical_image_manager.utils.validators import DataValidator

# Variables globales del sistema
image_manager = None
data_loader = None
validator = None

def inicializar_sistema():
    """Inicializar los componentes principales del sistema."""
    global image_manager, data_loader, validator
    
    image_manager = ImageManager(storage_path="../data/notebook_demo")
    data_loader = DataLoader(download_dir="../data/notebook_downloads")
    validator = DataValidator()
    
    print("Sistema inicializado correctamente")
    print(f"ImageManager: {len(image_manager)} imágenes registradas")
    print("DataLoader configurado")
    print("DataValidator listo")

# Ejecutar inicialización
inicializar_sistema()

Sistema inicializado correctamente
ImageManager: 0 imágenes registradas
DataLoader configurado
DataValidator listo


In [2]:
# Esta celda ya no es necesaria - el notebook ahora funciona con archivos reales
# como en main.py, donde el usuario proporciona rutas de archivos

print("Sistema listo para trabajar con archivos de imagen reales.")
print("Use la función 'registrar_imagen()' para agregar imágenes desde rutas de archivos.")

Sistema listo para trabajar con archivos de imagen reales.
Use la función 'registrar_imagen()' para agregar imágenes desde rutas de archivos.


## 2. Registrar Nueva Imagen

Función para registrar una imagen médica desde archivo (igual que main.py)

In [4]:
def create_metadata_interactive():
    """Crear metadatos de forma interactiva."""
    metadata = {}
    
    print("\n--- Agregar Metadatos Personalizados ---")
    while True:
        key = input("Ingrese clave de metadato (Enter para terminar): ").strip()
        if not key:
            break
        value = input(f"Ingrese valor para '{key}': ").strip()
        
        # Intentar convertir a tipo apropiado
        try:
            if value.isdigit():
                value = int(value)
            elif '.' in value and value.replace('.', '').replace('-', '').isdigit():
                value = float(value)
        except:
            pass
        
        metadata[key] = value
    
    return metadata if metadata else None

def registrar_imagen():
    """Registrar una nueva imagen en el sistema (igual que main.py)."""
    global image_manager
    
    print("\n--- Registrar Nueva Imagen ---")
    
    file_path = input("Ingrese la ruta del archivo de imagen: ").strip()
    
    if not file_path:
        print("La ruta del archivo no puede estar vacía.")
        return
    
    if not os.path.exists(file_path):
        print(f"Archivo no encontrado: {file_path}")
        return
    
    try:
        # Preguntar si el usuario quiere agregar metadatos
        add_metadata = input("¿Agregar metadatos personalizados? (s/n): ").strip().lower() == 's'
        
        metadata = None
        if add_metadata:
            metadata = create_metadata_interactive()
        
        # Registrar la imagen
        load_pixels = input("¿Cargar datos de píxeles en memoria? (s/n): ").strip().lower() == 's'
        image_id = image_manager.register_image(
            file_path=file_path,
            metadata=metadata,
            load_pixel_data=load_pixels
        )
        
        print(f"\n¡Imagen registrada exitosamente!")
        print(f"ID de Imagen: {image_id}")
        
    except Exception as e:
        print(f"Error al registrar imagen: {e}")

# Ejecutar función de registro
registrar_imagen()


--- Registrar Nueva Imagen ---


Ingrese la ruta del archivo de imagen:  C:\Users\EYNAR\Desktop\MAESTRAI_IA_MOD1\data\test\TestImage.jpg
¿Agregar metadatos personalizados? (s/n):  n
¿Cargar datos de píxeles en memoria? (s/n):  n


ERROR:medical_image_manager.utils.file_handler:opencv-python library not available


Error al registrar imagen: Standard image support requires opencv-python library


Imagen registrada exitosamente:
  Archivo: chest_ct.dcm
  ID: 55b121ac-2dd1-443a-a02e-77bb9042f9da
  Modalidad: CT
  Dimensiones: (512, 512, 100)


Seleccione el número de archivo a registrar (1-3):  1


Imagen registrada exitosamente:
  Archivo: brain_mri_t1.nii
  ID: 7e3d4a43-058d-4ca9-828a-7b602c46c899
  Modalidad: MRI
  Dimensiones: (128, 128, 64)


## 3. Listar Imágenes Registradas

Mostrar todas las imágenes actualmente registradas en el sistema

In [12]:
def listar_imagenes():
    """Mostrar lista de todas las imágenes registradas."""
    global image_manager
    
    if not image_manager.images:
        print("No hay imágenes registradas en el sistema")
        return
    
    print(f"Total de imágenes registradas: {len(image_manager.images)}")
    print("-" * 60)
    
    for i, (image_id, imagen) in enumerate(image_manager.images.items(), 1):
        print(f"{i}. Archivo: {imagen.filename}")
        print(f"   ID: {image_id}")
        
        # Corregir el uso de get_metadata
        modalidad = imagen.get_metadata('modality')
        print(f"   Modalidad: {modalidad if modalidad else 'No especificada'}")
        
        print(f"   Formato: {imagen.format_type}")
        print(f"   Tamaño: {imagen.file_size} bytes" if imagen.file_size else "   Tamaño: No disponible")
        print(f"   Fecha creación: {imagen.creation_date.strftime('%Y-%m-%d %H:%M:%S')}")
        
        if imagen.pixel_data is not None:
            print(f"   Dimensiones: {imagen.pixel_data.shape}")
        else:
            print("   Dimensiones: Datos no cargados")
        
        print()

# Ejecutar listado de imágenes
listar_imagenes()

No hay imágenes registradas en el sistema


## 4. Buscar Imágenes

Buscar imágenes por criterios específicos

In [13]:
def buscar_imagenes():
    """Buscar imágenes por diferentes criterios."""
    global image_manager
    
    if not image_manager.images:
        print("No hay imágenes registradas para buscar")
        return
    
    print("Opciones de búsqueda:")
    print("1. Buscar por modalidad")
    print("2. Buscar por ID de paciente")
    print("3. Buscar por nombre de archivo")
    
    try:
        opcion = input("Seleccione opción de búsqueda (1-3): ")
        
        if opcion == "1":
            modalidad = input("Ingrese modalidad (MRI, CT, etc.): ")
            resultados = image_manager.search_images(modalidad)
            
        elif opcion == "2":
            paciente_id = input("Ingrese ID de paciente: ")
            resultados = image_manager.search_images(paciente_id)
            
        elif opcion == "3":
            nombre_archivo = input("Ingrese nombre de archivo: ")
            resultados = image_manager.search_images(nombre_archivo)
            
        else:
            print("Opción inválida")
            return
        
        if resultados:
            print(f"Se encontraron {len(resultados)} resultado(s):")
            print("-" * 40)
            
            for imagen in resultados:
                print(f"Archivo: {imagen.filename}")
                print(f"ID: {imagen.image_id}")
                
                # Corregir el uso de get_metadata
                modalidad = imagen.get_metadata('modality')
                paciente_id = imagen.get_metadata('patient_id')
                
                print(f"Modalidad: {modalidad if modalidad else 'No especificada'}")
                print(f"Paciente: {paciente_id if paciente_id else 'No especificado'}")
                print()
        else:
            print("No se encontraron imágenes que coincidan con el criterio de búsqueda")
            
    except KeyboardInterrupt:
        print("Búsqueda cancelada")

# Ejecutar búsqueda
buscar_imagenes()

No hay imágenes registradas para buscar


No se encontraron imágenes que coincidan con el criterio de búsqueda


## 5. Ver Información de Imagen Específica

Mostrar información detallada de una imagen seleccionada

In [14]:
def ver_informacion_imagen():
    """Mostrar información detallada de una imagen específica."""
    global image_manager
    
    if not image_manager.images:
        print("No hay imágenes registradas")
        return
    
    print("Imágenes disponibles:")
    imagenes_lista = list(image_manager.images.items())
    
    for i, (image_id, imagen) in enumerate(imagenes_lista, 1):
        print(f"{i}. {imagen.filename} (ID: {image_id[:8]}...)")
    
    try:
        opcion = input(f"Seleccione imagen (1-{len(imagenes_lista)}): ")
        indice = int(opcion) - 1
        
        if 0 <= indice < len(imagenes_lista):
            image_id, imagen = imagenes_lista[indice]
            
            print(f"\nInformación detallada de: {imagen.filename}")
            print("=" * 50)
            print(f"ID de imagen: {image_id}")
            print(f"Nombre de archivo: {imagen.filename}")
            print(f"Formato: {imagen.format_type}")
            print(f"Fecha de creación: {imagen.creation_date.strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"Última modificación: {imagen.last_modified.strftime('%Y-%m-%d %H:%M:%S')}")
            
            if imagen.file_size:
                print(f"Tamaño de archivo: {imagen.file_size} bytes")
            
            if imagen.pixel_data is not None:
                print(f"Dimensiones: {imagen.pixel_data.shape}")
                print(f"Tipo de datos: {imagen.pixel_data.dtype}")
                print(f"Valor mínimo: {np.min(imagen.pixel_data)}")
                print(f"Valor máximo: {np.max(imagen.pixel_data)}")
                print(f"Valor medio: {np.mean(imagen.pixel_data):.2f}")
            
            print("\nMetadatos:")
            if imagen.metadata:
                for clave, valor in imagen.metadata.items():
                    print(f"  {clave}: {valor}")
            else:
                print("  No hay metadatos disponibles")
                
        else:
            print("Opción inválida")
            
    except (ValueError, KeyboardInterrupt):
        print("Operación cancelada")

# Ejecutar visualización de información
ver_informacion_imagen()

No hay imágenes registradas



Información detallada de: chest_ct.dcm
ID de imagen: 11efb532-adb8-4e11-a75a-7497702618c8
Nombre de archivo: chest_ct.dcm
Formato: DICOM
Fecha de creación: 2025-10-18 21:12:04
Última modificación: 2025-10-18 21:12:04
Dimensiones: (512, 512, 100)
Tipo de datos: uint16
Valor mínimo: 0
Valor máximo: 65535
Valor medio: 16608.15

Metadatos:
  modality: CT
  body_part: CHEST
  patient_id: PAT002
  study_date: 2024-01-16
  kvp: 120
  slice_thickness: 0.625


## 6. Modificar Metadatos de Imagen

Actualizar metadatos de una imagen existente

In [15]:
def modificar_metadatos():
    """Modificar metadatos de una imagen existente."""
    global image_manager
    
    if not image_manager.images:
        print("No hay imágenes registradas")
        return
    
    print("Seleccione imagen para modificar metadatos:")
    imagenes_lista = list(image_manager.images.items())
    
    for i, (image_id, imagen) in enumerate(imagenes_lista, 1):
        print(f"{i}. {imagen.filename}")
    
    try:
        opcion = input(f"Seleccione imagen (1-{len(imagenes_lista)}): ")
        indice = int(opcion) - 1
        
        if 0 <= indice < len(imagenes_lista):
            image_id, imagen = imagenes_lista[indice]
            
            print(f"\nModificando metadatos de: {imagen.filename}")
            print("Metadatos actuales:")
            for clave, valor in imagen.metadata.items():
                print(f"  {clave}: {valor}")
            
            print("\nOpciones de modificación:")
            print("1. Agregar/Actualizar campo de metadatos")
            print("2. Eliminar campo de metadatos")
            
            mod_opcion = input("Seleccione opción (1-2): ").strip()
            
            if mod_opcion == '1':
                clave = input("Ingrese clave del metadato: ").strip()
                valor = input("Ingrese valor del metadato: ").strip()
                
                # Intentar convertir a tipo apropiado
                try:
                    if valor.isdigit():
                        valor = int(valor)
                    elif '.' in valor and valor.replace('.', '').isdigit():
                        valor = float(valor)
                except:
                    pass
                
                exito = image_manager.update_image_metadata(image_id, {clave: valor})
                
                if exito:
                    print(f"Metadato actualizado: {clave} = {valor}")
                else:
                    print("Error al actualizar metadato")
            
            elif mod_opcion == '2':
                if imagen.metadata:
                    print("\nClaves de metadatos existentes:")
                    claves = list(imagen.metadata.keys())
                    for i, clave in enumerate(claves, 1):
                        print(f"{i}. {clave}")
                    
                    try:
                        opcion_clave = int(input(f"Seleccione clave a eliminar (1-{len(claves)}): ")) - 1
                        if 0 <= opcion_clave < len(claves):
                            clave_eliminar = claves[opcion_clave]
                            imagen.remove_metadata(clave_eliminar)
                            print(f"Eliminada clave de metadato: {clave_eliminar}")
                        else:
                            print("Número de clave inválido")
                    except ValueError:
                        print("Por favor ingrese un número válido")
                else:
                    print("No hay metadatos para eliminar")
                
        else:
            print("Opción inválida")
            
    except (ValueError, KeyboardInterrupt):
        print("Operación cancelada")

# Ejecutar modificación de metadatos
modificar_metadatos()

No hay imágenes registradas


## 7. Eliminar Imagen

Eliminar una imagen del sistema

In [16]:
def eliminar_imagen():
    """Eliminar una imagen del sistema."""
    global image_manager
    
    if not image_manager.images:
        print("No hay imágenes registradas")
        return
    
    print("Seleccione imagen para eliminar:")
    imagenes_lista = list(image_manager.images.items())
    
    for i, (image_id, imagen) in enumerate(imagenes_lista, 1):
        print(f"{i}. {imagen.filename} (ID: {image_id[:8]}...)")
    
    try:
        opcion = input(f"Seleccione imagen a eliminar (1-{len(imagenes_lista)}): ")
        indice = int(opcion) - 1
        
        if 0 <= indice < len(imagenes_lista):
            image_id, imagen = imagenes_lista[indice]
            
            print(f"\nAdvertencia: Se eliminará la imagen '{imagen.filename}'")
            confirmacion = input("¿Está seguro? (y/N): ").lower().strip()
            
            if confirmacion == 'y':
                eliminada = image_manager.delete_image(image_id)
                
                if eliminada:
                    print("Imagen eliminada exitosamente")
                else:
                    print("Error al eliminar la imagen")
            else:
                print("Eliminación cancelada")
                
        else:
            print("Opción inválida")
            
    except (ValueError, KeyboardInterrupt):
        print("Operación cancelada")

# Ejecutar eliminación de imagen
eliminar_imagen()

No hay imágenes registradas


## 8. Menú Principal Interactivo

Sistema de menú completo como en main.py. Para ejecutar el menú interactivo, descomente la última línea de la celda siguiente.

In [17]:
def mostrar_menu():
    """Mostrar opciones del menú principal."""
    print("\n" + "="*50)
    print("            MENÚ PRINCIPAL")
    print("="*50)
    print("1. Registrar Nueva Imagen")
    print("2. Ver Imágenes Registradas")
    print("3. Modificar Metadatos de Imagen")
    print("4. Eliminar Imagen")
    print(" extras ------")
    print("5. Buscar Imágenes")
    print("6. Cargar Dataset de Zenodo (descargar)")
    print("7. Salir")
    print("="*50)

def ejecutar_menu_principal():
    """Ejecutar el menú principal del sistema."""
    
    # Mapeo de funciones - igual que main.py
    funciones_menu = {
        '1': registrar_imagen,
        '2': listar_imagenes,
        '3': modificar_metadatos,
        '4': eliminar_imagen,
        '5': buscar_imagenes,
        '6': lambda: print("Funcionalidad de carga de dataset no implementada en notebook"),
    }
    
    while True:
        try:
            mostrar_menu()
            opcion = input("Seleccione una opción (1-7): ").strip()
            
            if opcion == '7':
                print("\n¡ Gracias Magister !")
                break
            
            elif opcion in funciones_menu:
                print(f"\nEjecutando opción {opcion}...")
                funciones_menu[opcion]()
                input("\nPresione Enter para continuar...")
                
            else:
                print("Opción inválida. Por favor intente de nuevo.")
                
        except KeyboardInterrupt:
            print("\n\nAplicación interrumpida por el usuario. ¡Hasta luego!")
            break
        except Exception as e:
            print(f"Error inesperado: {e}")
            print("Continuando...")

# Ejecutar el menú principal
ejecutar_menu_principal()


            MENÚ PRINCIPAL
1. Registrar Nueva Imagen
2. Ver Imágenes Registradas
3. Modificar Metadatos de Imagen
4. Eliminar Imagen
 extras ------
5. Buscar Imágenes
6. Cargar Dataset de Zenodo (descargar)
7. Salir


Seleccione una opción (1-7):  1



Ejecutando opción 1...

--- Registrar Nueva Imagen ---


Ingrese la ruta del archivo de imagen:  C:\Users\EYNAR\Desktop\MAESTRAI_IA_MOD1\data\test\TestImage.jpg
¿Agregar metadatos personalizados? (s/n):  n
¿Cargar datos de píxeles en memoria? (s/n):  n


ERROR:medical_image_manager.utils.file_handler:opencv-python library not available


...

Opción inválida



Presione Enter para continuar... 1



            MENÚ PRINCIPAL
1. Registrar Nueva Imagen
2. Ver Imágenes Registradas
3. Modificar Metadatos de Imagen
4. Eliminar Imagen
 extras ------
5. Buscar Imágenes
6. Cargar Dataset de Zenodo (descargar)
7. Salir


Seleccione una opción (1-7):  1



Ejecutando opción 1...
Opciones disponibles para registro:
1. brain_mri_t1.nii
2. chest_ct.dcm
3. spine_mri_t2.nii.gz
